# Vertex AI RAG Pipeline: Re-indexing with `text-embedding-005`

This notebook reinitializes and reindexes the Acura MDX 2022 Owner’s Manual using the latest Vertex AI embedding model: `text-embedding-005`. 

We'll use:
- `sentence-transformers` for local testing
- `text-embedding-005` for indexing on Vertex AI
- Vertex Matching Engine for storage & search
- Gemini for answering queries

This setup ensures that we use a consistent 768-dimensional embedding across both index and queries, preventing dimensional mismatch errors from previous runs.

In [ ]:
# 🧱 Cell 1: Install packages, load environment, and initialize Vertex AI

# Uncomment if packages haven't been installed yet
# !pip install --upgrade google-cloud-aiplatform python-dotenv vertexai

import os
from dotenv import load_dotenv, find_dotenv
from google.cloud import aiplatform
import vertexai

# Load .env and set ADC (Application Default Credentials)
load_dotenv(find_dotenv(), override=True)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GCP_KEY_PATH")

# Initialize Vertex AI
project_id = os.getenv("GCP_PROJECT_ID")
region = os.getenv("VERTEX_REGION")
vertexai.init(project=project_id, location=region)
aiplatform.init(project=project_id, location=region)

print(f"✅ Vertex AI initialized with project: {project_id}, region: {region}")

In [ ]:
# 🧱 Cell 2: Load and chunk the Acura MDX 2022 user manual PDF

import fitz  # PyMuPDF
import nltk
from pathlib import Path
from nltk.tokenize import PunktSentenceTokenizer

# Load environment variable for the PDF
pdf_local_path = os.getenv("PDF_LOCAL_PATH")
pdf_path = Path(pdf_local_path)

# Download tokenizer model
print("📦 Downloading NLTK punkt model...")
nltk.download("punkt")

# Explicitly load tokenizer
tokenizer = PunktSentenceTokenizer()

# Open the PDF
doc = fitz.open(pdf_path)
print(f"📄 Loaded PDF: {pdf_path.name} with {len(doc)} pages")

# Extract text and tokenize into sentences
all_sentences = []
for i, page in enumerate(doc):
    text = page.get_text()
    sentences = tokenizer.tokenize(text)
    all_sentences.extend(sentences)
print(f"🧠 Extracted {len(all_sentences)} sentences from all pages")

# Group into ~500-word chunks
chunks = []
chunk = ""
for sentence in all_sentences:
    if len(chunk.split()) + len(sentence.split()) <= 500:
        chunk += " " + sentence
    else:
        chunks.append(chunk.strip())
        chunk = sentence
if chunk:
    chunks.append(chunk.strip())

print(f"✅ Final chunk count: {len(chunks)} (each ≤ ~500 words)")

In [ ]:
# 🧱 Cell 3: Generate embeddings for PDF chunks using text-embedding-005

from vertexai.preview.language_models import TextEmbeddingModel
import tqdm

# Initialize the embedding model (768-dim output)
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-005")

# Embed chunks in batches to avoid memory issues
embedded_chunks = []
batch_size = 20  # tweak if needed
for i in tqdm.trange(0, len(chunks), batch_size, desc="🔢 Embedding chunks"):
    batch = chunks[i:i+batch_size]
    embeddings = embedding_model.get_embeddings(batch)
    embedded_chunks.extend([e.values for e in embeddings])

print(f"✅ Embedded {len(embedded_chunks)} chunks.")

In [ ]:
# 🧱 Cell 4 (Updated): Create new Vertex AI Tree-AH Index with 768-dim and neighbors count

from google.cloud import aiplatform

# Init Vertex AI SDK
aiplatform.init(project=os.getenv("GCP_PROJECT_ID"), location=os.getenv("VERTEX_REGION"))

new_index_display_name = "acura-mdx-index-v3-768-dim"
dimension = 768

# ✅ Create index with required approximateNeighborsCount
vs_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=new_index_display_name,
    dimensions=dimension,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    shard_size="SHARD_SIZE_SMALL",
    index_update_method="STREAM_UPDATE",
    approximate_neighbors_count=150,  # Required for tree-AH
)

print(f"✅ Index created: {vs_index.display_name}")
print(f"🔗 Resource name: {vs_index.resource_name}")

In [ ]:
# 🧱 Cell 5: Create a new public Vertex AI Matching Engine endpoint

from google.cloud import aiplatform

# Define display name for the endpoint
endpoint_display_name = "acura-mdx-endpoint-v3-768-dim"

# Create the endpoint (public)
vs_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=endpoint_display_name,
    public_endpoint_enabled=True,
)

print(f"✅ Endpoint created: {vs_endpoint.display_name}")
print(f"🔗 Resource name: {vs_endpoint.resource_name}")

In [ ]:
# 🧱 Cell 6: Deploy index to endpoint with versioned index ID
#LONG Running process !!!

# Already available: vs_index and vs_endpoint from previous cells

deployed_index_id = "acura_mdx_v3_768_dim"

vs_deployed_index = vs_endpoint.deploy_index(
    index=vs_index,
    deployed_index_id=deployed_index_id,
    display_name=vs_index.display_name,
    machine_type="e2-standard-16",
    min_replica_count=1,
    max_replica_count=1,
)

print(f"✅ Deployed index '{vs_index.display_name}' to endpoint '{vs_endpoint.display_name}'")
print(f"🔗 Deployed Index ID: {deployed_index_id}")

In [ ]:
load_dotenv(find_dotenv(), override=True)
gcs_bucket_name_v2 = os.getenv("GCS_BUCKET_NAME_V2")
print(f"gcs_bucket_name_v2: {gcs_bucket_name_v2}")

In [ ]:
# 🧱 Cell 7A: Upload to Vertex Vector Index using LlamaIndex with explicit credentials

from google.oauth2 import service_account
from llama_index.embeddings.vertex import VertexTextEmbedding
from llama_index.vector_stores.vertexaivectorsearch import VertexAIVectorStore
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext

# Load from .env
load_dotenv(find_dotenv(), override=True)
project_id = os.getenv("GCP_PROJECT_ID")
gcs_bucket_name_v2 = os.getenv("GCS_BUCKET_NAME_V2")

# Load Vertex credentials securely
key_path = os.getenv("GCP_KEY_PATH")
credentials = service_account.Credentials.from_service_account_file(key_path)

# Init embedding model
embed_model = VertexTextEmbedding(
    model_name="text-embedding-005",
    project=project_id,
    location="us-central1",
    credentials=credentials,
)

# Setup vector store
vector_store = VertexAIVectorStore(
    project_id=project_id,
    region="us-central1",
    index_id=vs_index.resource_name,
    endpoint_id=vs_endpoint.resource_name,
    gcs_bucket_name=gcs_bucket_name_v2,
)

# Convert chunks to TextNodes
nodes = []
for i, (text, embedding) in enumerate(zip(chunks, embedded_chunks)):
    node = TextNode(
        id_=f"chunk-{i}",
        text=text,
        embedding=embedding,
        metadata={"source": "Acura MDX Manual", "chunk_index": i}
    )
    nodes.append(node)

# Upload to vector store
vector_store.add(nodes)

print(f"✅ Uploaded {len(nodes)} chunks to Vertex AI Vector Store using LlamaIndex")

In [ ]:
# !pip install "google-cloud-aiplatform[preview]<1.47"

In [54]:
# 🧱 Cell 8: Setup LlamaIndex RAG query engine (Vertex Vector Store + Gemini Pro)

from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.vertex import Vertex

# Initialize Gemini LLM for completion
llm = Vertex(
    model="gemini-pro",
    project=project_id,
    location="us-central1",
    credentials=credentials,
    temperature=0.2,
    max_tokens=1024,
)

# Assign global settings
Settings.llm = llm
Settings.embed_model = embed_model  # From previous cell

# Set up storage context from vector store
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index from storage
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model=embed_model
)

# Setup retriever-backed query engine
query_engine = index.as_query_engine()

# Ask your question!
response = query_engine.query("How do I adjust the seat in the 2022 Acura MDX?")

# Show result
print("📘 Gemini Response:")
print("-" * 80)
print(response.response)

C:\Users\prabu\AppData\Local\Temp\ipykernel_19092\2052847667.py:7: DeprecationWarning: Call to deprecated class Vertex. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Vertex(


📘 Gemini Response:
--------------------------------------------------------------------------------
To adjust the seat in the 2022 Acura MDX, follow these steps:

1. **Adjust the front power seats.**
    * Move the seat back to allow sufficient space between the steering wheel and your chest.
    * Adjust the seat-back angle to a comfortable, upright position.
    * Adjust the lumbar support, bolster support, and thigh support using the controls on the side of the seat.
2. **Adjust the second-row seats.**
    * Pull up the strap to change the angle of the seat-back.
    * Pull up the bar to move the seat forward or backward.
    * Pull up the lever to fold down the seat-back.

For more information on adjusting the seats, please refer to the Acura MDX Manual.


In [55]:
from crewai import LLM
from google.oauth2 import service_account
import os

# Load credentials
gcp_key_path = os.getenv("GCP_KEY_PATH")
gcp_project = os.getenv("GCP_PROJECT_ID")

# Secure credentials object
gcp_vertex_ai_credentials = service_account.Credentials.from_service_account_file(gcp_key_path)

# ✅ Initialize Gemini LLM via Vertex AI
gemini_llm = LLM(
    model='vertex_ai/gemini-2.0-flash',
    vertex_credentials=gcp_vertex_ai_credentials,
    vertex_project=gcp_project
)

print("✅ CrewAI Gemini LLM Initialized (via Vertex)")

✅ CrewAI Gemini LLM Initialized (via Vertex)


In [56]:
from crewai import Agent

# Agent 1: Weather Context Provider
weather_agent = Agent(
    role="Weather Agent",
    goal="Add relevant seasonal context for winter preparation.",
    backstory="A weather-focused AI agent specializing in identifying the effects of cold weather on vehicles.",
    llm=gemini_llm,
    verbose=True
)

# Agent 2: Acura Manual Expert
manual_review_agent = Agent(
    role="Manual Review Agent",
    goal="Review the Acura MDX 2022 owner's manual and find relevant information about seasonal or winter use.",
    backstory="An expert in parsing owner manuals to surface relevant operational and maintenance details.",
    llm=gemini_llm,
    verbose=True
)

# Agent 3: Vehicle Winterization Advisor
recommendation_agent = Agent(
    role="Recommendation Agent",
    goal="Suggest actions for winterizing a 2022 Acura MDX using Gemini and context from the manual.",
    backstory="A certified automotive assistant trained in seasonal readiness checks, with expertise in snow, battery care, tire selection, and cold weather maintenance.",
    llm=gemini_llm,
    verbose=True
)

print("✅ Crew AI Winterization Agents Successfully Defined")

✅ Crew AI Winterization Agents Successfully Defined


In [57]:
from crewai import Task

# Task 1: Add seasonal context (Winter)
add_seasonal_context_task = Task(
    description=(
        "Describe key winter weather conditions (cold, snow, ice, low visibility) "
        "that might affect a vehicle's operation, and provide a brief summary of why seasonal adjustments are necessary."
    ),
    agent=weather_agent,
    expected_output="A short winter-specific context paragraph about how weather impacts vehicle operation."
)

# Task 2: Review Acura MDX Manual for Winter Advice
manual_insights_task = Task(
    description=(
        "Based on the seasonal context, identify any winter-related maintenance or operational details from the 2022 Acura MDX owner's manual. "
        "This includes battery care, tire pressure, washer fluids, climate controls, and safety features that assist in cold weather."
    ),
    agent=manual_review_agent,
    expected_output="A bullet list of Acura MDX manual insights relevant to winterizing the vehicle."
)

# Task 3: Suggest Winterization Recommendations
generate_recommendations_task = Task(
    description=(
        "Using the seasonal context and manual insights, suggest a list of recommended steps to winterize a 2022 Acura MDX. "
        "Cover mechanical checks, safety checks, comfort features, and environmental preparations. Format in Markdown checklist."
    ),
    agent=recommendation_agent,
    expected_output="A Markdown-formatted winterization checklist for the Acura MDX."
)

print("✅ Winterization Tasks Successfully Defined")

✅ Winterization Tasks Successfully Defined


In [58]:
from crewai import Crew

# Create Crew AI pipeline
winterization_crew = Crew(
    agents=[weather_agent, manual_review_agent, recommendation_agent],
    tasks=[add_seasonal_context_task, manual_insights_task, generate_recommendations_task],
    verbose=True
)

# Run the workflow
print("🚀 Running Crew AI: Acura MDX Winterization Workflow...\n")
winterization_crew.kickoff()

🚀 Running Crew AI: Acura MDX Winterization Workflow...

# Agent: Weather Agent
## Task: Describe key winter weather conditions (cold, snow, ice, low visibility) that might affect a vehicle's operation, and provide a brief summary of why seasonal adjustments are necessary.


# Agent: Weather Agent
## Final Answer: 
Winter weather presents a unique set of challenges for vehicle operation. Cold temperatures, snow, ice, and reduced visibility can significantly impact safety and performance. Preparing your vehicle for these conditions is not merely a suggestion; it's a necessity for ensuring reliable transportation and preventing accidents throughout the winter months.

**Key Winter Weather Conditions and Their Effects on Vehicles:**

*   **Cold Temperatures:**
    *   **Battery Performance:** Cold reduces battery capacity, potentially leading to starting problems.
    *   **Oil Viscosity:** Oil thickens in cold, hindering lubrication and increasing engine wear.
    *   **Tire Pressure:** T

CrewOutput(raw='```markdown\n## 2022 Acura MDX Winterization Checklist\n\n### Mechanical Checks:\n\n- [ ] **Tires:**\n    - [ ] Install winter tires (same size and load range as original tires).\n    - [ ] Check tire pressure regularly (adjust for cold weather).\n    - [ ] Ensure all four tires are the same size, load range, speed rating, and construction.\n- [ ] **Battery:**\n    - [ ] Test battery health.\n    - [ ] Clean battery terminals.\n    - [ ] Consider using a battery maintainer if the vehicle is not driven for extended periods in cold weather.\n- [ ] **Coolant:**\n    - [ ] Check coolant level and antifreeze concentration.\n    - [ ] Ensure proper coolant/antifreeze mixture to prevent freezing.\n- [ ] **Washer Fluid:**\n    - [ ] Fill washer fluid reservoir with winter-specific washer fluid (antifreeze-equipped).\n- [ ] **Brakes:**\n    - [ ] Inspect brake pads and rotors.\n    - [ ] Check brake fluid level.\n- [ ] **Oil:**\n    - [ ] Consider using a lower viscosity oil app